# 🎨 Advanced Visualization – Geochemical Patterns & Pathfinders

**Project:** Gold Pathfinder ML Project  
**Notebook:** `03_visualization_advanced.ipynb`  
**Milestone:** 3 – Data Exploration / Visualization

This notebook extends the basic EDA with **richer visualizations** useful for:
- Understanding multi-element geochemical patterns
- Comparing sample types
- Highlighting potential pathfinder behaviour for gold (Au)

It uses the cleaned, merged dataset:

```text
1_datasets/processed/gold_assays_final.csv
```

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

pd.set_option('display.max_columns', 80)
pd.set_option('display.width', 160)
pd.set_option('display.float_format', lambda x: f'{x:0.4f}')
plt.rcParams['figure.figsize'] = (8, 5)

# Paths
PROJECT_ROOT = Path('..').resolve()
DATA_PROCESSED = PROJECT_ROOT / '1_datasets' / 'processed'
FIG_DIR = Path('.') / 'figures'
FIG_DIR.mkdir(exist_ok=True)
DATA_PROCESSED, FIG_DIR

## 1️⃣ Load Final Dataset

In [ ]:
final_path = DATA_PROCESSED / 'gold_assays_final.csv'
print(final_path, final_path.exists())
df = pd.read_csv(final_path)
df.head()

## 2️⃣ Focus on Gold and Pathfinder Elements

We define a subset of columns for detailed visualization.

In [ ]:
candidate_cols = ['au_ppm', 'as_ppm', 'sb_ppm', 'bi_ppm', 'cu_ppm', 'zn_ppm', 'pb_ppm', 'ag_ppm']
available = [c for c in candidate_cols if c in df.columns]
available

## 3️⃣ Pairwise Relationships (Seaborn Pairplot)

We visualize pairwise scatterplots and kernel density estimates for Au
and the most important pathfinders.

⚠️ Note: This can be slow if there are many rows; if needed, sample rows.

In [ ]:
# Optional: sample rows for speed
sample_size = 1500
plot_df = df[available].dropna()
if len(plot_df) > sample_size:
    plot_df = plot_df.sample(sample_size, random_state=42)
print('Using rows for pairplot:', len(plot_df))

g = sns.pairplot(plot_df, corner=True)
g.fig.suptitle('Pairwise Relationships – Au and Pathfinder Elements', y=1.02)
plt.tight_layout()
g.savefig(FIG_DIR / 'pairplot_au_pathfinders.png')
plt.show()

## 4️⃣ Gold vs Individual Pathfinder Elements

We create clearer scatterplots of Au vs each pathfinder with optional
log scales to handle skewness.

In [ ]:
for col in available:
    if col == 'au_ppm':
        continue
    plt.figure()
    sns.scatterplot(x=df[col], y=df['au_ppm'], alpha=0.6)
    plt.xlabel(col)
    plt.ylabel('au_ppm')
    plt.title(f'AU vs {col}')
    plt.tight_layout()
    plt.savefig(FIG_DIR / f'scatter_au_vs_{col}.png')
    plt.show()

### Optional: Log–Log Scatterplots

These plots can reveal trends hidden on linear scales.

In [ ]:
eps = 1e-6
for col in available:
    if col == 'au_ppm':
        continue
    x = df[col]
    y = df['au_ppm']
    valid = (x > 0) & (y > 0)
    if valid.sum() < 30:
        continue
    plt.figure()
    plt.scatter(np.log10(x[valid] + eps), np.log10(y[valid] + eps), alpha=0.5)
    plt.xlabel(f'log10({col})')
    plt.ylabel('log10(au_ppm)')
    plt.title(f'Log–Log Scatter: Au vs {col}')
    plt.tight_layout()
    plt.savefig(FIG_DIR / f'loglog_au_vs_{col}.png')
    plt.show()

## 5️⃣ Distributions by Sample Type

We compare how Au and key pathfinders behave in different sample types
(core, chip, trench, RC, grab) using boxplots.

In [ ]:
if 'sample_type' in df.columns:
    for col in ['au_ppm'] + [c for c in available if c != 'au_ppm']:
        plt.figure()
        sns.boxplot(data=df, x='sample_type', y=col)
        plt.title(f'{col} by Sample Type')
        plt.xticks(rotation=30)
        plt.tight_layout()
        plt.savefig(FIG_DIR / f'boxplot_{col}_by_sample_type.png')
        plt.show()
else:
    print('sample_type column not found; skipping sample-type comparisons.')

## 6️⃣ Heatmap for Selected Elements

We recompute a correlation matrix for a compact subset of elements
and generate a styled heatmap.

In [ ]:
sub = df[available].copy()
corr = sub.corr(method='pearson')
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, fmt='.2f', square=True, cbar_kws={'shrink': 0.8})
plt.title('Correlation Matrix – Au and Pathfinder Elements')
plt.tight_layout()
plt.savefig(FIG_DIR / 'correlation_heatmap_pathfinders.png')
plt.show()

## 7️⃣ Optional 3D Visualization (Plotly)

If spatial coordinates (`easting`, `northing`, `rl` / `elevation`) are available,
we can create a 3D scatterplot colored by gold grade.

In [ ]:
coord_cols = [c for c in ['easting', 'northing', 'rl', 'elevation', 'elevation_from', 'elevation_to'] if c in df.columns]
coord_cols

In [ ]:
if {'easting', 'northing'}.issubset(df.columns) and 'au_ppm' in df.columns:
    z_col = 'rl' if 'rl' in df.columns else ('elevation' if 'elevation' in df.columns else None)
    if z_col is not None:
        sample = df[['easting', 'northing', z_col, 'au_ppm']].dropna()
        sample_size = 3000
        if len(sample) > sample_size:
            sample = sample.sample(sample_size, random_state=42)
        fig = px.scatter_3d(
            sample,
            x='easting', y='northing', z=z_col,
            color='au_ppm',
            title='3D Distribution of Au (if coordinates available)',
        )
        fig.write_html(FIG_DIR / '3d_au_distribution.html')
        fig.show()
    else:
        print('Z-coordinate column not found; skipping 3D plot.')
else:
    print('Spatial columns not available; skipping 3D visualization.')

## ✅ Summary

This notebook generated **advanced visualizations** that complement the
basic EDA performed earlier:

- Pairplots to visualize pairwise relationships among Au and pathfinders
- Scatter and log–log plots of Au vs key elements
- Sample-type comparisons via boxplots
- Focused correlation heatmap for Au and candidate pathfinders
- Optional 3D visualization of Au if spatial coordinates exist

These plots will feed directly into:
- **Milestone 4 (Data Analysis & Modeling)** as justification for feature selection
- **Milestone 5 (Communication)** as candidate figures for the final report
  and stakeholder presentations.
